In [895]:
import numpy as np
from ANNModel import model as mod
from ANNModel import data as d
from PSO import pswarm as ps
from PSO import psobehaviour as psh
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_tanh.txt")
X = data.get_rows()
y = data.get_output()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [896]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = mod.ANNhistory(model)

Model Compiled!


In [897]:
model.dimension_vec()

[(0.0, 5.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (0.0, 5.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 1.0)]

In [898]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)

pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.assess_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)


[ 3.88731548  0.08783219 -0.55747126  0.9908425  -0.96565065  0.10648879
  0.33646417 -0.25375511 -0.81776308  2.02848897 -0.23881058  0.01216352
 -0.42755448 -0.71462912  0.96825774]
84.15843581694607


In [899]:
pso_best = pso.best.location
model.decode_vec(pso_best)

In [900]:
#model.layers[-1].bias = np.array([0.0])
#model.layers[-1].weights = np.array([[1.0]])


In [901]:
#run model
model.set_training_input(X_test, y_test)
model.one_pass()


In [902]:
model.y_hat

array([-0.97487126, -0.83537973, -0.90338913, -0.66997952,  0.287857  ,
        0.99314203,  0.99999894, -0.81120844,  1.        ,  0.94611252,
       -0.9979237 ,  1.        , -0.98734035,  0.99849867,  1.        ,
        0.98917566, -0.04759824, -0.99587997, -0.95451935, -0.99671494,
       -0.91510586,  0.92356393, -0.85625037, -0.98412498, -0.25172841,
        1.        , -0.99632034,  0.99914354,  0.17551892, -0.75078394,
       -0.97762001,  1.        ,  0.59262043,  0.06273789])

In [903]:
#score function
correctly_classified = 0


#get predictions
predictions = np.round(model.y_hat, 4)
print(model.y)
print(predictions)

#loop through predictions
for index, pred in enumerate(predictions):

    if np.isclose(y_test[index], pred):
        correctly_classified += 1

print('A,B: ', correctly_classified , " out of ", len(y_test))
correctly_classified = 0
#loop through predictions
for index, pred in enumerate(predictions):
    if np.isclose(pred, y_test[index]):
        correctly_classified += 1

print('B,A: ',correctly_classified , " out of ", len(y_test))

[-0.9999 -0.9757 -0.9951 -0.8337  0.7616  0.9993  1.     -0.964   1.
  0.9951 -1.      1.     -1.      0.9998  1.      0.999   0.3799 -1.
 -0.9996 -1.     -0.9967  0.9926 -0.9837 -1.      0.      1.     -1.
  0.9999  0.664  -0.9217 -1.      1.      0.9217  0.537 ]
[-0.9749 -0.8354 -0.9034 -0.67    0.2879  0.9931  1.     -0.8112  1.
  0.9461 -0.9979  1.     -0.9873  0.9985  1.      0.9892 -0.0476 -0.9959
 -0.9545 -0.9967 -0.9151  0.9236 -0.8563 -0.9841 -0.2517  1.     -0.9963
  0.9991  0.1755 -0.7508 -0.9776  1.      0.5926  0.0627]
A,B:  6  out of  34
B,A:  6  out of  34


In [904]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.HYPERBOLIC_TANGENT
bias:  [-0.23881058]
weights:  [[ 0.01216352]
 [-0.42755448]
 [-0.71462912]
 [ 0.96825774]]


In [905]:
model_history.historical_particle_fitness()
model_history.historical_particle_location()

In [906]:
len(model_history.vec_history[::10])

4020

In [907]:
len(model_history.particle_location[0][::10])

402